#Init

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
MODEL_DIR = '/content/drive/MyDrive/HeungEol/checkpoints (1)/08-11_060828_KoAlpaca_Dataset_v1.0_aloss2_lora3232_epoch3_bs8_maxlen_512_HeungEol_test_v2/checkpoints/epoch2 - loss0.1089'
INDUCING = False

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


#Load model & Some Samples

In [ ]:
!pip install numba

from numba import cuda
import torch
device = cuda.get_current_device()
device.reset()
cuda.close()
torch.cuda.is_available()

True

In [ ]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

In [ ]:
from peft import PeftModel, PeftConfig

In [ ]:
def load_peft_model(path):
    peft_model_id = path
    config = PeftConfig.from_pretrained(peft_model_id)
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map={"":0})
    model = PeftModel.from_pretrained(model, peft_model_id)
    tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

    model.eval()
    return model, tokenizer

In [ ]:
model, tokenizer = load_peft_model(MODEL_DIR)

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

In [ ]:
pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead',

In [ ]:
def gen(x):
    instruction = "다음 조건에 어울리는 가사를 생성하시오. 주어진 음절 수를 절대 벗어나지 말 것. 제목과 장르에 어울려야 할 것. 생성 형식은 ['가사 / 가사 / 가사 / 가사']와 같음."
    gen_notes = x['gen_notes']
    title = x['title']
    genre = x['genre']
    q = f"### Instruction(명령어):\n{instruction}\n\n### Input(입력):\n음절 수는 [{gen_notes}], 제목은 [{title}], 장르는 [{genre}]이다.\n\n### Response(응답): \n"
    print('input as follow:\n' + q)

    lyrics = pipe(
        q,
        do_sample=False,
        max_new_tokens=300,
        # temperature=0.5,
        # top_p=0.5,
        return_full_text=False,
        eos_token_id=2
    )
    return lyrics[0]['generated_text']

In [ ]:
import re

def gen_induced(x):
    instruction = "다음 조건에 어울리는 가사를 생성하시오. 주어진 음절 수를 절대 벗어나지 말 것. 제목과 장르에 어울려야 할 것. 생성 형식은 [(음절)가사 / (음절)가사 / (음절)가사 / (음절)가사]와 같음."
    gen_notes = x['gen_notes']
    title = x['title']
    genre = x['genre']
    q = f"### Instruction(명령어):\n{instruction}\n\n### Input(입력):\n음절 수는 [{gen_notes}], 제목은 [{title}], 장르는 [{genre}]이다.\n\n### Response(응답): \n"
    print('input as follow:\n' + q)

    lyrics = pipe(
        q,
        do_sample=True,
        max_new_tokens=300,
        temperature=0.75,
        top_p=0.5,
        return_full_text=False,
        eos_token_id=2
    )

    print(lyrics[0]['generated_text'])
    return re.sub('\([0-9]+\)','',lyrics[0]['generated_text'])

In [ ]:
x1 = {'gen_notes': '3 / 5 / 3', 'title': '짜증나는 너', 'genre': '록/메탈'}
x2 = {'gen_notes': '1 / 4 / 3 / 10 / 5 / 5 / 2 / 3 / 4', 'title': '너무 슬픈 날', 'genre': '발라드'}
x3 = {'gen_notes': '2 / 5 / 3 / 1 /4', 'title': '뻔한 사랑 노래', 'genre': '발라드'}

In [ ]:
if INDUCING:
    lyrics1 = gen_induced(x1)
    lyrics2 = gen_induced(x2)
    lyrics3 = gen_induced(x3)
else:
    lyrics1 = gen(x1)
    lyrics2 = gen(x2)
    lyrics3 = gen(x3)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


input as follow:
### Instruction(명령어):
다음 조건에 어울리는 가사를 생성하시오. 주어진 음절 수를 절대 벗어나지 말 것. 제목과 장르에 어울려야 할 것. 생성 형식은 ['가사 / 가사 / 가사 / 가사']와 같음.

### Input(입력):
음절 수는 [3 / 5 / 3], 제목은 [짜증나는 너], 장르는 [록/메탈]이다.

### Response(응답): 



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


input as follow:
### Instruction(명령어):
다음 조건에 어울리는 가사를 생성하시오. 주어진 음절 수를 절대 벗어나지 말 것. 제목과 장르에 어울려야 할 것. 생성 형식은 ['가사 / 가사 / 가사 / 가사']와 같음.

### Input(입력):
음절 수는 [1 / 4 / 3 / 10 / 5 / 5 / 2 / 3 / 4], 제목은 [너무 슬픈 날], 장르는 [발라드]이다.

### Response(응답): 



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


input as follow:
### Instruction(명령어):
다음 조건에 어울리는 가사를 생성하시오. 주어진 음절 수를 절대 벗어나지 말 것. 제목과 장르에 어울려야 할 것. 생성 형식은 ['가사 / 가사 / 가사 / 가사']와 같음.

### Input(입력):
음절 수는 [2 / 5 / 3 / 1 /4], 제목은 [뻔한 사랑 노래], 장르는 [발라드]이다.

### Response(응답): 



In [ ]:
lyrics1

'짜증나 / 짜증나는 너 / 짜증나'

In [ ]:
lyrics2

'내 / 마음까지 / 아프게 / 하는거야 / 너에게만은 / 너에게만은 / 너에게만은 / 너에게만은 / 너에게만은 / 너에게만은'

In [ ]:
lyrics3

'너를 / 사랑하니까 / 사랑해 / 널 / 사랑해'

#Make Evaluate Dataset
훈련 corpus에 없었던 가사 데이터 및 2022.4월 이후의 가사들 중 다양한 장르에 대해 샘플링

In [ ]:
import pandas as pd

eval_df = pd.read_csv('./drive/MyDrive/HeungEol/eval_dataset.csv', encoding = 'utf-8-sig', index_col = 0)

In [ ]:
eval_df.syls = eval_df.syls.apply(eval)

In [ ]:
eval_df

,title,genre,original_lyrics,syls
0,헤어지자 말해요,발라드,헤어지자고 말하려 오늘\n너에게 가다가 우리 추억 생각해 봤어\n처음 본 네 얼굴\...,"[6 / 9 / 8 / 7, 7 / 8 / 8 / 10 / 7 / 7 / 6 / 8..."
1,모래 알갱이,트로트,나는 작은 바람에도 흩어질\n나는 가벼운 모래 알갱이\n그대 이 모래에 작은 발걸음...,"[1 / 5 / 2 / 1, 1 / 8 / 7 / 4 / 9 / 2 / 6 / 10..."
2,부동의 첫사랑,인디음악,주인공은 아니었지\n누구의 시점에서도\n그냥 스쳐 지나가거나\n친구3이라거나\n이름...,"[10 / 4 / 8 / 5, 1 / 7 / 6 / 6 / 10 / 4 / 1 / ..."
3,건물 사이에 피어난 장미,인디음악,건물 사이에 피어난 장미\n제발 살아남아 줬으면\n꺾이지 마 잘 자라줘\n온몸을 덮...,"[1 / 6 / 7 / 5, 8 / 9 / 5 / 10 / 5 / 9 / 2 / 5..."
4,잠깐 시간 될까,인디음악,잠깐 시간 될까\n만날 수 있을까\n별일은 아니고 그냥 보고 싶어 그래\n웃지 말고...,"[7 / 1 / 6 / 4, 10 / 4 / 7 / 6 / 4 / 5 / 9 / 2..."
5,운이 좋았지,발라드,나는 운이 좋았지\n다른 사람들은\n그렇게 어려운 이별을 한다는데\n나는 운이 좋았...,"[6 / 1 / 9 / 6, 2 / 10 / 4 / 9 / 4 / 2 / 9 / 7..."
6,그건 아마 우리의 잘못은 아닐 거야,R&B,사실은 나도 잘 모르겠어\n불안한 마음은 어디에서 태어나\n우리에게까지 온 건지\n...,"[7 / 1 / 5 / 1, 2 / 6 / 5 / 9 / 4 / 1 / 9 / 9,..."
7,지나간 여름을 안타까워마,인디음악,내 여름을 너에게 맡길게\n저 푸르른 잎도 언젠가는 녹슬어버리고\n우릴 감싸는 이 ...,"[4 / 2 / 2 / 1, 9 / 1 / 6 / 2 / 7 / 4 / 2 / 8,..."
8,어떤 위로,록/메탈,널 많이 좋아한다고\n더 많이 말했어야 했는데\n불안에 떨며 울먹이던 너\n난 너를...,"[5 / 6 / 10 / 6, 10 / 9 / 1 / 4 / 5 / 4 / 9 / ..."
9,주제는 사랑,록/메탈,다른 말로 풀 수 없는\n돌려 말하고 싶진 않은\n헤맬 수밖에 없는\n의미를 알고 ...,"[1 / 2 / 5 / 8, 5 / 8 / 9 / 1 / 6 / 6 / 2 / 6,..."


In [ ]:
generated_lines = [[gen_induced({'gen_notes':gen_notes, 'title':row['title'], 'genre':row['genre']}) if INDUCING else gen({'gen_notes':gen_notes, 'title':row['title'], 'genre':row['genre']}) for gen_notes in row['syls']] for idx, row in eval_df.iterrows()]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


input as follow:
### Instruction(명령어):
다음 조건에 어울리는 가사를 생성하시오. 주어진 음절 수를 절대 벗어나지 말 것. 제목과 장르에 어울려야 할 것. 생성 형식은 ['가사 / 가사 / 가사 / 가사']와 같음.

### Input(입력):
음절 수는 [6 / 9 / 8 / 7], 제목은 [헤어지자 말해요], 장르는 [발라드]이다.

### Response(응답): 



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


input as follow:
### Instruction(명령어):
다음 조건에 어울리는 가사를 생성하시오. 주어진 음절 수를 절대 벗어나지 말 것. 제목과 장르에 어울려야 할 것. 생성 형식은 ['가사 / 가사 / 가사 / 가사']와 같음.

### Input(입력):
음절 수는 [7 / 8 / 8 / 10 / 7 / 7 / 6 / 8], 제목은 [헤어지자 말해요], 장르는 [발라드]이다.

### Response(응답): 



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

input as follow:
### Instruction(명령어):
다음 조건에 어울리는 가사를 생성하시오. 주어진 음절 수를 절대 벗어나지 말 것. 제목과 장르에 어울려야 할 것. 생성 형식은 ['가사 / 가사 / 가사 / 가사']와 같음.

### Input(입력):
음절 수는 [1 / 2 / 5 / 8], 제목은 [주제는 사랑], 장르는 [록/메탈]이다.

### Response(응답): 



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


input as follow:
### Instruction(명령어):
다음 조건에 어울리는 가사를 생성하시오. 주어진 음절 수를 절대 벗어나지 말 것. 제목과 장르에 어울려야 할 것. 생성 형식은 ['가사 / 가사 / 가사 / 가사']와 같음.

### Input(입력):
음절 수는 [5 / 8 / 9 / 1 / 6 / 6 / 2 / 6], 제목은 [주제는 사랑], 장르는 [록/메탈]이다.

### Response(응답): 



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


input as follow:
### Instruction(명령어):
다음 조건에 어울리는 가사를 생성하시오. 주어진 음절 수를 절대 벗어나지 말 것. 제목과 장르에 어울려야 할 것. 생성 형식은 ['가사 / 가사 / 가사 / 가사']와 같음.

### Input(입력):
음절 수는 [2 / 9 / 7 / 6 / 10 / 3 / 3 / 2 / 1 / 8 / 4 / 7], 제목은 [주제는 사랑], 장르는 [록/메탈]이다.

### Response(응답): 



In [ ]:
eval_df['generated_lines'] = generated_lines

In [ ]:
eval_df

,title,genre,original_lyrics,syls,generated_lines
0,헤어지자 말해요,발라드,헤어지자고 말하려 오늘\n너에게 가다가 우리 추억 생각해 봤어\n처음 본 네 얼굴\...,"[6 / 9 / 8 / 7, 7 / 8 / 8 / 10 / 7 / 7 / 6 / 8...",[헤어지자 말해요 / 그대가 없는 세상에서 / 나는 살 수가 없죠 / 사랑하는 그대...
1,모래 알갱이,트로트,나는 작은 바람에도 흩어질\n나는 가벼운 모래 알갱이\n그대 이 모래에 작은 발걸음...,"[1 / 5 / 2 / 1, 1 / 8 / 7 / 4 / 9 / 2 / 6 / 10...","[그 / 모래알갱이 / 같은 / 너, 난 / 너에게로 달려가고 / 있는 것 같은데 ..."
2,부동의 첫사랑,인디음악,주인공은 아니었지\n누구의 시점에서도\n그냥 스쳐 지나가거나\n친구3이라거나\n이름...,"[10 / 4 / 8 / 5, 1 / 7 / 6 / 6 / 10 / 4 / 1 / ...","[내가 너를 사랑하는 이유 / 너를 사랑 / 하는 이유는 너라서 / 너라서 너라서,..."
3,건물 사이에 피어난 장미,인디음악,건물 사이에 피어난 장미\n제발 살아남아 줬으면\n꺾이지 마 잘 자라줘\n온몸을 덮...,"[1 / 6 / 7 / 5, 8 / 9 / 5 / 10 / 5 / 9 / 2 / 5...","[난 / 너에게로 달려가 / 너에게로 달려가 / 너에게로 가, 건물 사이에 피어난 ..."
4,잠깐 시간 될까,인디음악,잠깐 시간 될까\n만날 수 있을까\n별일은 아니고 그냥 보고 싶어 그래\n웃지 말고...,"[7 / 1 / 6 / 4, 10 / 4 / 7 / 6 / 4 / 5 / 9 / 2...","[잠깐 시간 될까 / 나 / 너와 함께 있고 / 싶은 것 같아, 잠깐 시간 될까 우..."
5,운이 좋았지,발라드,나는 운이 좋았지\n다른 사람들은\n그렇게 어려운 이별을 한다는데\n나는 운이 좋았...,"[6 / 1 / 9 / 6, 2 / 10 / 4 / 9 / 4 / 2 / 9 / 7...","[나는 운이 좋지 / 난 / 운이 좋지 난 난 운이 좋지 / 난 운이 좋지, 나는 ..."
6,그건 아마 우리의 잘못은 아닐 거야,R&B,사실은 나도 잘 모르겠어\n불안한 마음은 어디에서 태어나\n우리에게까지 온 건지\n...,"[7 / 1 / 5 / 1, 2 / 6 / 5 / 9 / 4 / 1 / 9 / 9,...","[우리가 사랑했던 / 그 / 시간들 속에 / 난, 그건 / 아마 우리의 잘못은 / ..."
7,지나간 여름을 안타까워마,인디음악,내 여름을 너에게 맡길게\n저 푸르른 잎도 언젠가는 녹슬어버리고\n우릴 감싸는 이 ...,"[4 / 2 / 2 / 1, 9 / 1 / 6 / 2 / 7 / 4 / 2 / 8,...","[지나간 / 여름 / 속에 / 난, 지나간 여름을 아쉬워 / 마 / 지나간 여름을 ..."
8,어떤 위로,록/메탈,널 많이 좋아한다고\n더 많이 말했어야 했는데\n불안에 떨며 울먹이던 너\n난 너를...,"[5 / 6 / 10 / 6, 10 / 9 / 1 / 4 / 5 / 4 / 9 / ...","[내가 있잖아 / 내가 있잖아 / 내가 있잖아 내가 있잖아 / 내가 있잖아, 그대가..."
9,주제는 사랑,록/메탈,다른 말로 풀 수 없는\n돌려 말하고 싶진 않은\n헤맬 수밖에 없는\n의미를 알고 ...,"[1 / 2 / 5 / 8, 5 / 8 / 9 / 1 / 6 / 6 / 2 / 6,...","[내 / 맘을 / 받아줄래요 / 그대와의 / 사랑을 시작해요, 사랑이란 / 말로는 ..."


#Align Metric
주어진 마디 k개에 대하여, 해당 마디의 음절을 맞추었다면 1, 아니면 0. 이후 최종적인 점수는 마디 개수 k개로 나누어 Normalize

##Function

In [ ]:
def lyrics_align_metric(generated_line, original_line, notes=True):
    """
    Calculate aligning score for a given pair of lyrics.

    Args:
    - generated_line (str): The lyrics from the generated line.
    - original_line (str): The original line of lyrics.

    Returns:
    - score (float): From 0(if all bars are wrongfully aligned) to 1(if all bars are successfully aligned)
    """

    gen_syls = [len(words.replace(' ','')) for words in generated_line.split(' / ')]
    if not notes:
        orig_syls = [len(words) for words in original_line.split(' / ')]
    else:
        orig_syls = [int(note) for note in original_line.split(' / ')]

    # Pad 0 to match the length
    while len(gen_syls) < len(orig_syls):
        gen_syls.append(0)
    while len(orig_syls) < len(gen_syls):
        orig_syls.append(0)

    score = 0

    for g, o in zip(gen_syls, orig_syls):
        if g == o:
            score += 1

    return score / len(gen_syls)

##Sample

In [ ]:
lyrics_align_metric('랄랄라 / 랄랄랄라 / 랄 / 랄라', '딴딴따 / 딴딴딴따 / 딴 / 따란', notes=False)

1.0

In [ ]:
lyrics_align_metric('랄랄라 / 랄랄랄라 / 랄', '딴딴따 / 딴딴딴따 / 딴 / 따란', notes=False)

0.75

In [ ]:
lyrics_align_metric('랄랄라 / 랄랄랄라 / 랄 / 라라랄', '딴딴따 / 딴딴딴따 / 딴 / 따란', notes=False)

0.75

In [ ]:
lyrics_align_metric('랄랄라 / 랄 / 랄 / 랄라', '딴딴따 / 딴딴딴따 / 딴 / 따란', notes=False)

0.75

In [ ]:
lyrics_align_metric('랄라 / 랄라 / 랄라 / 랄라', '딴딴따 / 딴딴딴따 / 딴 / 따란', notes=False)

0.25

In [ ]:
lyrics_align_metric('사랑은 다 끝났어 / 나를 떠나 행복해줘 난 / 널 잊을 수 있을 거야 / 사랑은 다 끝났어', '6 / 9 / 8 / 7')

0.75

##Evaluate eval_set

In [ ]:
align_scores = []
for _, row in eval_df.iterrows():
    align_scores.append([lyrics_align_metric(g,s) for g,s in zip(row['generated_lines'], row['syls'])])

In [ ]:
eval_df['align_scores'] = align_scores

In [ ]:
eval_df

,title,genre,original_lyrics,syls,generated_lines,align_scores
0,헤어지자 말해요,발라드,헤어지자고 말하려 오늘\n너에게 가다가 우리 추억 생각해 봤어\n처음 본 네 얼굴\...,"[6 / 9 / 8 / 7, 7 / 8 / 8 / 10 / 7 / 7 / 6 / 8...",[헤어지자 말해요 / 그대가 없는 세상에서 / 나는 살 수가 없죠 / 사랑하는 그대...,"[0.5, 0.125, 0.16666666666666666]"
1,모래 알갱이,트로트,나는 작은 바람에도 흩어질\n나는 가벼운 모래 알갱이\n그대 이 모래에 작은 발걸음...,"[1 / 5 / 2 / 1, 1 / 8 / 7 / 4 / 9 / 2 / 6 / 10...","[그 / 모래알갱이 / 같은 / 너, 난 / 너에게로 달려가고 / 있는 것 같은데 ...","[1.0, 0.625, 0.3333333333333333]"
2,부동의 첫사랑,인디음악,주인공은 아니었지\n누구의 시점에서도\n그냥 스쳐 지나가거나\n친구3이라거나\n이름...,"[10 / 4 / 8 / 5, 1 / 7 / 6 / 6 / 10 / 4 / 1 / ...","[내가 너를 사랑하는 이유 / 너를 사랑 / 하는 이유는 너라서 / 너라서 너라서,...","[0.75, 0.125, 0.46153846153846156]"
3,건물 사이에 피어난 장미,인디음악,건물 사이에 피어난 장미\n제발 살아남아 줬으면\n꺾이지 마 잘 자라줘\n온몸을 덮...,"[1 / 6 / 7 / 5, 8 / 9 / 5 / 10 / 5 / 9 / 2 / 5...","[난 / 너에게로 달려가 / 너에게로 달려가 / 너에게로 가, 건물 사이에 피어난 ...","[0.75, 0.2222222222222222, 0.5]"
4,잠깐 시간 될까,인디음악,잠깐 시간 될까\n만날 수 있을까\n별일은 아니고 그냥 보고 싶어 그래\n웃지 말고...,"[7 / 1 / 6 / 4, 10 / 4 / 7 / 6 / 4 / 5 / 9 / 2...","[잠깐 시간 될까 / 나 / 너와 함께 있고 / 싶은 것 같아, 잠깐 시간 될까 우...","[0.5, 0.5, 0.25]"
5,운이 좋았지,발라드,나는 운이 좋았지\n다른 사람들은\n그렇게 어려운 이별을 한다는데\n나는 운이 좋았...,"[6 / 1 / 9 / 6, 2 / 10 / 4 / 9 / 4 / 2 / 9 / 7...","[나는 운이 좋지 / 난 / 운이 좋지 난 난 운이 좋지 / 난 운이 좋지, 나는 ...","[0.5, 0.375, 0.4]"
6,그건 아마 우리의 잘못은 아닐 거야,R&B,사실은 나도 잘 모르겠어\n불안한 마음은 어디에서 태어나\n우리에게까지 온 건지\n...,"[7 / 1 / 5 / 1, 2 / 6 / 5 / 9 / 4 / 1 / 9 / 9,...","[우리가 사랑했던 / 그 / 시간들 속에 / 난, 그건 / 아마 우리의 잘못은 / ...","[1.0, 0.25, 0.08333333333333333]"
7,지나간 여름을 안타까워마,인디음악,내 여름을 너에게 맡길게\n저 푸르른 잎도 언젠가는 녹슬어버리고\n우릴 감싸는 이 ...,"[4 / 2 / 2 / 1, 9 / 1 / 6 / 2 / 7 / 4 / 2 / 8,...","[지나간 / 여름 / 속에 / 난, 지나간 여름을 아쉬워 / 마 / 지나간 여름을 ...","[0.75, 0.3333333333333333, 0.3333333333333333]"
8,어떤 위로,록/메탈,널 많이 좋아한다고\n더 많이 말했어야 했는데\n불안에 떨며 울먹이던 너\n난 너를...,"[5 / 6 / 10 / 6, 10 / 9 / 1 / 4 / 5 / 4 / 9 / ...","[내가 있잖아 / 내가 있잖아 / 내가 있잖아 내가 있잖아 / 내가 있잖아, 그대가...","[0.5, 0.2222222222222222, 0.25]"
9,주제는 사랑,록/메탈,다른 말로 풀 수 없는\n돌려 말하고 싶진 않은\n헤맬 수밖에 없는\n의미를 알고 ...,"[1 / 2 / 5 / 8, 5 / 8 / 9 / 1 / 6 / 6 / 2 / 6,...","[내 / 맘을 / 받아줄래요 / 그대와의 / 사랑을 시작해요, 사랑이란 / 말로는 ...","[0.6, 0.125, 0.25]"


#Semantic Metric
생성된 가사가 다양성은 유지하면서, 원본 가사와 얼마나 유사하게 생성되었는지 판단하는 지표  

##Function

In [ ]:
import pandas as pd

In [ ]:
corpus = list(pd.read_csv('/content/drive/MyDrive/HeungEol/HeungEol_Dataset_v1.0.csv', encoding='utf-8-sig')['가사'])

In [ ]:
corpus[:5]

['J 스치는 바람에 J 그대 모습 보이면\n난 오늘도 조용히 그댈 그리워하네\nJ 지난밤 꿈속에 J 만났던 모습은\n내 가슴 속깊이 여울져 남아 있네\n\nJ 아름다운 여름날이 멀리 사라졌다 해도\nJ 나의 사랑은 아직도 변함 없는데\nj 난 너를 못잊어 J 난 너를 사랑해\n\nJ 우리가 걸었던 J 추억의 그 길을\n난 이밤도 쓸쓸히 쓸쓸히 걷고 있네\n\nJ 아름다운 여름날이 멀리 사라졌다 해도\nJ 나의 사랑은 아직도 변함 없는데\nj 난 너를 못잊어 J 난 너를 사랑해\n\nJ 우리가 걸었던 J 추억의 그길을\n난 이밤도 쓸쓸히 쓸쓸히 걷고 있네\n\n쓸쓸히 걷고 있네 -',
 '나는 나는 갯바위\n당신은 나를 사랑하는 파도\n어느 고운 바람 불던날\n잔잔히 다가와\n부드러운 손길로 나를 감싸고\n향기로운 입술도 내게 주었지\n세찬 비바람에 내몸이 패이고\n이는 파도에 깨끗이 부서져도\n나의 생은 당신의\n조각품인것을\n나는 당신으로 인해\n아름다운 것을\n나는 나는 갯바위\n당신은 나를 사랑하는 파도\n우린 오늘도 마주보며\n이렇게 서있네\n\n세찬 비바람에 내몸이 패이고\n이는 파도에 깨끗이 부서져도\n나의 생은 당신의\n조각품인것을\n나는 당신으로 인해\n아름다운 것을\n나는 나는 갯바위\n당신은 나를 사랑하는 파도\n우린 오늘도 마주보며\n이렇게 서있네\n나는 나는 갯바위\n당신은 나를 사랑하는 파도\n우린 오늘도 마주보며\n이렇게 서있네\n이렇게 서있네',
 '별이 유난히도 밝은\n오늘 이 시간이 가면\n그대\n떠난다~는 말이\n나를 슬프~게 하네\n이 밤 다가도록\n아 행복했~던 시절\n많은 우리들~의 약속\n자꾸 귓가를 스쳐\n나를 슬프~게 하네\n그대 잘못 아니예요\n왠지 울고 싶어져요\n나는 너무나~도 파란\n꿈을 꾸고 있었어요\n\n별이 유난히도 밝은\n오늘 이 시간이 가면\n그대\n떠난다~는 말이\n나를 슬프~게 하네\n이 밤 다가도록\n아 행복했~던 시절\n많은 우리들~의 약속\n자꾸 귓가를 스쳐\n나를 슬프~게 하네\n그대 잘못

In [ ]:
len(corpus)

28569

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)

df = (vectorizer.transform(corpus) > 0).sum(axis=0).tolist()[0]
inverted_normalized_df = [1 - (value/len(corpus))*2 for value in df]

inverted_normalized_df_dict = dict(zip(vectorizer.get_feature_names_out(), inverted_normalized_df))

In [ ]:
sorted(inverted_normalized_df_dict.items(), key = lambda x: x[1])[:75]

[('내가', 0.4436627113304631),
 ('나를', 0.4965172039623368),
 ('너의', 0.5010675907452133),
 ('나의', 0.5640029402499213),
 ('나는', 0.5768140291924814),
 ('너를', 0.5868948860653156),
 ('없는', 0.5969057369876439),
 ('you', 0.6069165879099723),
 ('다시', 0.6129370996534705),
 ('내게', 0.6497602296195177),
 ('우리', 0.6513003605306451),
 ('너무', 0.6667016696419196),
 ('있는', 0.7218663586404844),
 ('있어', 0.7285869298890406),
 ('없어', 0.7314571738597782),
 ('이젠', 0.7386677867618747),
 ('함께', 0.7407679652770485),
 ('모든', 0.7416780426336238),
 ('너와', 0.7459484056144772),
 ('이제', 0.7479785781791453),
 ('그대', 0.7524589590115159),
 ('이렇게', 0.7571493576954041),
 ('모두', 0.7582694529034968),
 ('my', 0.7589695124085547),
 ('같은', 0.7613497147257517),
 ('me', 0.763589905141937),
 ('지금', 0.7649200182015471),
 ('the', 0.7654100598550877),
 ('속에', 0.7726906787076901),
 ('네가', 0.7794112499562462),
 ('거야', 0.7806013511148447),
 ('to', 0.7887920473240225),
 ('않아', 0.7894921068290804),
 ('it', 0.7897021246805979),
 ('눈을', 0.79

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge

def calculate_triplet(word_gen, word_orig, inverted_normalized_df_dict):
    """
    Calculate the triplet (ROUGE score, tf-idf score, checked status) for a given pair of words.

    Args:
    - word_gen (str): The word from the generated line.
    - word_orig (str): The word from the original lyrics.
    - inverted_normalized_df_dict (dict): A dictionary containing the tf-idf scores for each word in the original lyrics.

    Returns:
    - tuple: A triplet containing the ROUGE score, tf-idf score, and checked status.
    """

    word_gen_spaced = ' '.join(word_gen)
    word_orig_spaced = ' '.join(word_orig)

    # Calculate ROUGE score
    rouge = Rouge()
    scores = rouge.get_scores(word_gen_spaced, word_orig_spaced)
    rouge_score = scores[0]['rouge-1']['r']

    # Get tf-idf score
    tfidf_score = inverted_normalized_df_dict.get(word_orig, 0.0)

    # Set checked status (initially set to False)
    checked = False

    return (rouge_score, tfidf_score, checked)


In [ ]:
import numpy as np

def lyrics_similarity_metric(generated_line, original_lyrics, inverted_normalized_df_dict):
    original_words = original_lyrics.split()
    generated_words = generated_line.split()

    # Construct the score matrix using list comprehensions
    scores_matrix = np.array([[calculate_triplet(w_g, w_o, inverted_normalized_df_dict) for w_o in original_words] for w_g in generated_words])

    # Determine penalty_diverse
    max_indices = np.argmax(scores_matrix[:,:,0], axis=1)
    penalty_diverse = len(max_indices) - len(np.unique(max_indices))

    scores = []

    for _ in range(len(generated_words)):
        # Find the highest score in the matrix
        i, j = np.unravel_index(np.argmax(scores_matrix[:,:,0], axis=None), scores_matrix[:,:,0].shape)
        max_score = scores_matrix[i, j, 0]
        scores.append(max_score * scores_matrix[i, j, 1])

        # Set the entire corresponding row and column to -1
        scores_matrix[i, :, 0] = -1
        scores_matrix[:, j, 0] = -1

    # Calculate the final score
    final_score = (sum(scores) / len(scores)) * (((len(generated_words) - penalty_diverse) / len(generated_words))*0.5 + 0.5)

    return final_score

##Sample

In [ ]:
calculate_triplet('오늘마저', '오늘도', inverted_normalized_df_dict)

(0.6666666666666666, 0.8620882775035879, False)

In [ ]:
rouge = Rouge()
rouge.get_scores('오 늘 마 저', '오 늘 도')

[{'rouge-1': {'r': 0.6666666666666666, 'p': 0.5, 'f': 0.5714285665306124},
  'rouge-2': {'r': 0.5, 'p': 0.3333333333333333, 'f': 0.39999999520000007},
  'rouge-l': {'r': 0.6666666666666666, 'p': 0.5, 'f': 0.5714285665306124}}]

In [ ]:
GT = """그땐 난 어떤 마음이었길래
내 모든걸 주고도 웃을 수 있었나 yeah yeah yeah yeah
그대는 또 어떤 마음이었길래
그 모든걸 갖고도 돌아서 버렸나 yeah yeah yeah yeah
뜨거운 여름밤은 가고
남은 건 볼 품 없지만
또다시 찾아오는 누군갈 위해서 남겨두겠소
다짐은 세워올린 모래성은
심술이 또 터지면 무너지겠지만 yeah yeah yeah yeah
뜨거운 여름밤은 가고
남은 건 볼 품 없지만
또다시 찾아오는
누군갈 위해서 남겨두겠소
그리운 그 마음 그대로
영원히 담아둘거야
언젠가 불어오는 바람에
남몰래 날려보겠소
눈이 부시던 그 순간들도
가슴아픈 그대의 거짓말도
새하얗게 바래지고
비틀거리던 내 발걸음도
그늘아래 드리운 내 눈빛도
아름답게 피어나길
눈이 부시던 그 순간들도
가슴아픈 그대의 거짓말도
새하얗게 바래지고
비틀거리던 내 발걸음도
그늘아래 드리운 내 눈빛도
아름답게 피어나길
눈이 부시던 그 순간들도
가슴아픈 그대의 거짓말도
새하얗게 바래지고
비틀거리던 내 발걸음도
그늘아래 드리운 내 눈빛도"""

In [ ]:
lyrics_similarity_metric('뜨거웠던 여름밤 난 바래질 모든 순간을 남겨둘 수 있으리',GT, inverted_normalized_df_dict)

0.42580309141091816

In [ ]:
lyrics_similarity_metric('여름밤은 뜨겁다 또 하얗게 바랠 누군갈 아프게 또 남겨요',GT, inverted_normalized_df_dict)

0.45649470482768817

In [ ]:
lyrics_similarity_metric('뜨겁게도 하얗게 널 사랑해 만날 누군갈 사랑할 수 있을까',GT, inverted_normalized_df_dict)

0.3367187368333715

In [ ]:
lyrics_similarity_metric('뜨거운 여름밤은 뜨거운 여름밤은 뜨거운 여름밤은',GT, inverted_normalized_df_dict)

0.5248923010307728

In [ ]:
lyrics_similarity_metric('뜨거운 여름밤은 뜨거운 여름밤에 뜨겁게 지새우리',GT, inverted_normalized_df_dict)

0.5303563302880745

##Evaluate eval_set

In [ ]:
semantic_scores = []
for _, row in eval_df.iterrows():
    semantic_scores.append([lyrics_similarity_metric(g, row['original_lyrics'], inverted_normalized_df_dict) for g in row['generated_lines']])

In [ ]:
semantic_scores

[[0.24948769454927477, 0.18767598215889678, 0.22555161188701034],
 [0.1587729599873132, 0.11046224929118974, 0.041937151915530546],
 [0.23150717811013938, 0.12272591503611141, 0.10432221043772716],
 [0.14956765491733465, 0.2920731660628653, 0.20155039793273946],
 [0.3325467674614275, 0.2912437090024624, 0.19005319187482497],
 [0.2684581216747174, 0.3859237066400645, 0.19612446367656372],
 [0.19780843923173802, 0.33438990707404836, 0.29143631239720463],
 [0.23501991716914986, 0.21965062169173274, 0.24139269099778243],
 [0.08891818920979092, 0.10735799075659529, 0.08935761321846465],
 [0.196820417235465, 0.15876129874315942, 0.1305121226797437]]

In [ ]:
eval_df['semantic_scores'] = semantic_scores

#Result

In [ ]:
print(f"{'Generated Line':<50} {'Align Score':<15} {'Semantic Score':<15}")
print('-' * 80)  # print a separator line

for _, row in eval_df.iterrows():
    for g, a, s in zip(row['generated_lines'], row['align_scores'], row['semantic_scores']):
        # Truncate the generated line if it's too long
        g = (g[:47] + '...') if len(g) > 50 else g
        print(f"{g:<50} {a:<15.2f} {s:<15.2f}")
    print('-' * 80)  # print a separator line


Generated Line                                     Align Score     Semantic Score 
--------------------------------------------------------------------------------
헤어지자 말해요 / 그대가 없는 세상에서 / 나는 살 수가 없죠 / 사랑하는 그대여     0.50            0.25           
내가 싫어졌나요 / 아니면 내가 미워졌니 / 아니면 내가 싫어졌니 / 아니면 내가 미... 0.12            0.19           
헤어지자 / 말해요 그대에게 / 내가 싫어졌다 / 말해요 그대에게 / 내가 지겨워졌다... 0.17            0.23           
--------------------------------------------------------------------------------
그 / 모래알갱이 / 같은 / 너                                 1.00            0.16           
난 / 너에게로 달려가고 / 있는 것 같은데 / 너는 나를 / 스쳐가고 있는 것 같아... 0.62            0.11           
사랑하는 너 / 너 / 너 / 너와 / 함께 걷던 바닷가에 / 너 / 너 / 너와 함... 0.33            0.04           
--------------------------------------------------------------------------------
내가 너를 사랑하는 이유 / 너를 사랑 / 하는 이유는 너라서 / 너라서 너라서       0.75            0.23           
난 / 너에게만은 특별해 / 너에게만은 / 너에게만은 / 너에게만은 / 너에게만은 특... 0.12            0.12           
너와 / 나 / 사

In [ ]:
eval_df['align_scores'].apply(lambda x: sum(x)/3).mean()

0.42603276353276354

In [ ]:
eval_df['semantic_scores'].apply(lambda x: sum(x)/3).mean()

0.20104698843403562

In [ ]:
with open('model2.txt', 'w') as f:
    f.write('align_scores: '+ str(eval_df['align_scores'].apply(lambda x: sum(x)/3).mean())+'\nsemantic_scores: '+ str(eval_df['semantic_scores'].apply(lambda x: sum(x)/3).mean()))